# WallStreetBests: Reddit Sentiment $\implies$ Stock Alpha

In [ ]:
import pandas as pd

# Adjust path in order to access preprocessing code
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)
import src.preprocess as preprocess

## 1: Quick data audit

In [19]:
# Read in Reddit data as pd.DataFrame
reddit = preprocess.load_reddit()

### 1a: Null counts per column

In [12]:
# For each column
for col in reddit.columns:
    # Count returns the number of non null values in an object
    print(f'{col}: {reddit[col].count()}')

title: 53187
score: 53187
id: 53187
url: 53187
comms_num: 53187
created: 53187
body: 24738
timestamp: 53187


### 1b: Top-10 tickers by post count and by up-vote score.